# 패키지 및 데이터 불러오기 불러오기

In [27]:
import re
import pandas as pd
import numpy
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize


DEFAULT_PATH='./data_in/'
train_df = pd.read_csv(DEFAULT_PATH+"labeledTrainData.tsv", header=0, delimiter="\t", quoting=3)

In [28]:
reviews = list(train_df['review'])
sentiments = list(train_df['sentiment'])

# 전처리 작업

In [29]:
def preprocessing( review, remove_stopwords = False ): 
    # 불용어 제거는 옵션으로 선택 가능하다.
    # 1. HTML 태그 제거
    review_text = BeautifulSoup(review, "html5lib").get_text()
    # 2. 영어가 아닌 특수문자들을 공백(" ")으로 바꾸기
    review_text = re.sub("[^a-zA-Z]", " ", review_text)
    # 3. 대문자들을 소문자로 바꾸고 공백단위로 텍스트들 나눠서 리스트로 만든다.
    words = review_text.lower().split()
    if remove_stopwords: 
        # 4. 불용어들을 제거
    
        #영어에 관련된 불용어 불러오기
        stops = set(stopwords.words("english"))
        # 불용어가 아닌 단어들로 이루어진 새로운 리스트 생성
        words = [w for w in words if not w in stops]
        # 5. 단어 리스트를 공백을 넣어서 하나의 글로 합친다.
        clean_review = ' '.join(words)
    else: # 불용어 제거하지 않을 때
        clean_review = ' '.join(words)

    return clean_review

In [30]:
clean_train_reviews = []
for review in reviews:
    clean_train_reviews.append(preprocessing(review, remove_stopwords = True))

# CountVectorizer를 활용한 벡터화

In [31]:
from sklearn.feature_extraction.text import CountVectorizer

In [32]:
#scikit-learn의 CountVectorizer는 단어의 출현 빈도수를 통해 벡터화를 한다.
vectorizer = CountVectorizer(analyzer = "word", # 단어에 대해서 분석 한다.
                             tokenizer = None, # 이미 단어로 구분되어 있으므로 None 
                             preprocessor = None, # 전처리를 하지 않는다.
                             stop_words = None, # 불용어 처리 하지 않는다.
                             max_features = 5000) # 최대 빈도수 5000까지 사용 
# 우리의 데이터를 선언 함수에 넣어서 벡터화 한다.
train_data_features = vectorizer.fit_transform(clean_train_reviews)

In [33]:
train_data_features

<25000x5000 sparse matrix of type '<class 'numpy.int64'>'
	with 1975048 stored elements in Compressed Sparse Row format>

In [ ]:
train_data_features = train_data_features.toarray()
print(type(train_data_features)) # 넘파이 배열로 변경 되었는지 확인
print(train_data_features.shape)

<class 'numpy.ndarray'>
(25000, 5000)


# 학습과 검증 데이터 분리

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
import numpy

y = train_df['sentiment']

# 학습 데이터와 테스트 데이터 분류한다.
train_input, eval_input, train_label, eval_label = train_test_split(train_data_features, y, test_size=0.2, random_state=42)

# 모델 구현 및 학습 

In [ ]:
from sklearn.ensemble import RandomForestClassifier


# 랜덤 포레스트 분류기에  100개 의사 결정 트리를 사용한다.
forest = RandomForestClassifier(n_estimators = 100) 

# 단어 묶음을 벡터화한 데이터와 정답 데이터를 가지고 학습을 시작한다.
forest = forest.fit( train_input, train_label )

In [ ]:
print("Accuracy: %f" % forest.score(eval_input, eval_label))  # 검증함수로 정확도 측정

Accuracy: 0.844200


# 검증 데이터셋으로 성능 평가

In [ ]:
test_df = pd.read_csv(DEFAULT_PATH + 'testData.tsv', header=0, delimiter='\t', quoting=3)

clean_test_reviews = []
for review in test_df['review']:
    clean_test_reviews.append(preprocessing(review, remove_stopwords = True))

test_sentences = []
for review in clean_test_reviews:
    test_sentences.append(review.split())

In [ ]:
test_data_features = vectorizer.transform(clean_test_reviews)
test_data_features = test_data_features.toarray()

# 데이터 제출하기

In [ ]:
# 위에서 만든 랜덤 포레스트 분류기를 통해 예측값을 가져온다.
result = forest.predict(test_data_features)

# 판다스 데이터 프레임을 통해 데이터를 구성해서 output에 넣는다.
output = pd.DataFrame( data={"id":test_df["id"], "sentiment":result} )

# 이제 csv파일로 만든다.
output.to_csv( "Bag_of_Words_model.csv", index=False, quoting=3 )